In [1]:
from IPython.display import Markdown, Latex
from datasets import load_dataset
import numpy as np
import pandas as pd

In [2]:
methods = [
    "danjacobellis/LSDIR_demucs_orig",
    # "danjacobellis/LSDIR_demucs_2xRR",
    # "danjacobellis/LSDIR_demucs_2x_walloc",
    # "danjacobellis/LSDIR_demucs_5xRR",
    # "danjacobellis/LSDIR_demucs_5x_walloc",
    "danjacobellis/LSDIR_demucs_10xRR",
    "danjacobellis/LSDIR_demucs_10x_walloc",
]

In [3]:
method_aggregated = {}

for method in methods:
    ds = load_dataset(method, split='validation')
    df = ds.to_pandas()
    method_aggregated[method] = df.mean()

In [4]:
results_df = pd.DataFrame(method_aggregated)
results_df = results_df.transpose()

In [5]:
results_df

,SSDR_other,SSDR_drums,SSDR_bass,SSDR_vocals,SRDR_other,SRDR_drums,SRDR_bass,SRDR_vocals,PSNR_other,PSNR_drums,PSNR_bass,PSNR_vocals,CDPAM_dB_other,CDPAM_dB_drums,CDPAM_dB_bass,CDPAM_dB_vocals
danjacobellis/LSDIR_demucs_orig,13.753074,22.302781,15.660749,14.246989,5.007309,9.913120,0.768342,-0.350839,37.382034,40.244511,40.930796,41.371783,4.705127,4.786927,4.951358,4.920378
danjacobellis/LSDIR_demucs_10xRR,10.704823,16.199578,14.704946,6.966281,2.198909,5.626160,-1.134002,-6.824239,35.039341,37.322374,40.381729,37.350482,4.355985,4.126468,4.931243,4.275728
danjacobellis/LSDIR_demucs_10x_walloc,13.210964,15.545982,15.413898,8.713092,2.591630,5.371202,-0.345901,-4.613902,34.775065,37.343891,39.475253,38.148983,4.582460,4.108161,4.833813,4.511948


In [6]:
print(results_df.to_markdown())

|                                       |   SSDR_other |   SSDR_drums |   SSDR_bass |   SSDR_vocals |   SRDR_other |   SRDR_drums |   SRDR_bass |   SRDR_vocals |   PSNR_other |   PSNR_drums |   PSNR_bass |   PSNR_vocals |   CDPAM_dB_other |   CDPAM_dB_drums |   CDPAM_dB_bass |   CDPAM_dB_vocals |
|:--------------------------------------|-------------:|-------------:|------------:|--------------:|-------------:|-------------:|------------:|--------------:|-------------:|-------------:|------------:|--------------:|-----------------:|-----------------:|----------------:|------------------:|
| danjacobellis/LSDIR_demucs_orig       |      13.7531 |      22.3028 |     15.6607 |      14.247   |      5.00731 |      9.91312 |    0.768342 |     -0.350839 |      37.382  |      40.2445 |     40.9308 |       41.3718 |          4.70513 |          4.78693 |         4.95136 |           4.92038 |
| danjacobellis/LSDIR_demucs_10xRR      |      10.7048 |      16.1996 |     14.7049 |       6.96628 |     

In [7]:
metrics = ['SSDR', 'SRDR', 'PSNR', 'CDPAM_dB']
instruments = ['other', 'drums', 'bass', 'vocals']
df_metrics = pd.DataFrame(index=results_df.index)
for metric in metrics:
    cols = [f"{metric}_{instrument}" for instrument in instruments]
    df_metrics[metric] = results_df[cols].mean(axis=1)

In [8]:
df_metrics

,SSDR,SRDR,PSNR,CDPAM_dB
danjacobellis/LSDIR_demucs_orig,16.490898,3.834483,39.982281,4.840948
danjacobellis/LSDIR_demucs_10xRR,12.143907,-0.033293,37.523481,4.422356
danjacobellis/LSDIR_demucs_10x_walloc,13.220984,0.750757,37.435798,4.509095


In [9]:
scores = pd.DataFrame(index=results_df.index, columns=['other', 'drums', 'bass', 'vocals'])
instruments = ['other', 'drums', 'bass', 'vocals']
metrics = ['SSDR', 'SRDR', 'PSNR', 'CDPAM_dB']

# Compute scores
for instrument in instruments:
    total_weights = 0
    score = pd.Series(0, index=results_df.index)
    for metric in metrics:
        metric_name = f'{metric}_{instrument}'
        first_row_value = results_df.loc['danjacobellis/LSDIR_demucs_orig', metric_name]
        
        # Skip if the first row value is zero or negative (to avoid division by zero or negative weights)
        if first_row_value <= 0:
            continue
        
        weight = 0.25 / first_row_value
        total_weights += weight
        
        # Calculate the weighted metric for each model
        score += weight * results_df[metric_name]
    
    # Normalize the score to ensure the original signal has a score of 1
    score /= score.loc['danjacobellis/LSDIR_demucs_orig']
    scores[instrument] = score.round(4)

In [10]:
scores

,other,drums,bass,vocals
danjacobellis/LSDIR_demucs_orig,1.0000,1.0000,1.0000,1.0000
danjacobellis/LSDIR_demucs_10xRR,0.7702,0.7708,0.3614,0.7536
danjacobellis/LSDIR_demucs_10x_walloc,0.8456,0.7562,0.6187,0.8169


In [11]:
df_combined = pd.concat([scores, df_metrics], axis=1)
cols = df_combined.columns.tolist()
cols.insert(3, cols.pop(0))
df_combined = df_combined[cols]

In [12]:
df_combined

,drums,bass,vocals,other,SSDR,SRDR,PSNR,CDPAM_dB
danjacobellis/LSDIR_demucs_orig,1.0000,1.0000,1.0000,1.0000,16.490898,3.834483,39.982281,4.840948
danjacobellis/LSDIR_demucs_10xRR,0.7708,0.3614,0.7536,0.7702,12.143907,-0.033293,37.523481,4.422356
danjacobellis/LSDIR_demucs_10x_walloc,0.7562,0.6187,0.8169,0.8456,13.220984,0.750757,37.435798,4.509095


In [13]:
df_rounded = df_combined.round(5)
column_format = 'l' + 'rr' * len(metrics)
latex_table = df_rounded.to_latex(
    index=True,
    float_format="%.3g",
    column_format=column_format,
    multicolumn=True,
    multicolumn_format='c',
    bold_rows=True
)
print(latex_table)

\begin{tabular}{lrrrrrrrr}
\toprule
 & drums & bass & vocals & other & SSDR & SRDR & PSNR & CDPAM_dB \\
\midrule
\textbf{danjacobellis/LSDIR_demucs_orig} & 1 & 1 & 1 & 1 & 16.5 & 3.83 & 40 & 4.84 \\
\textbf{danjacobellis/LSDIR_demucs_10xRR} & 0.771 & 0.361 & 0.754 & 0.77 & 12.1 & -0.0333 & 37.5 & 4.42 \\
\textbf{danjacobellis/LSDIR_demucs_10x_walloc} & 0.756 & 0.619 & 0.817 & 0.846 & 13.2 & 0.751 & 37.4 & 4.51 \\
\bottomrule
\end{tabular}

